In [14]:
from scipy.io import netcdf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import netCDF4 as nc
import matplotlib as mpl
import xarray as xr
import geopandas as gpd

In [15]:
# install modul yang diperlukan
import pandas as pd
import xarray as xr
from netCDF4 import Dataset
import numpy as np

In [16]:
#script ini digunakan untuk melakukan re-gridding menjadi 0.05 derajat

In [17]:
ds = xr.open_dataset("pm25.nc") #membukan file .nc yang diperlukan
ds

<xarray.Dataset>
Dimensions:    (longitude: 3, latitude: 3, time: 60)
Coordinates:
  * longitude  (longitude) float32 106.0 106.8 107.5
  * latitude   (latitude) float32 -5.5 -6.25 -7.0
  * time       (time) datetime64[ns] 2018-01-01 2018-02-01 ... 2022-12-01
Data variables:
    pm2p5      (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-11-22 08:52:07 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [18]:
latii = [-6.1556594]
lonii = [106.8388152]

In [19]:
# menentukan longitude baru
# menentukan latitude baru
new_lon = np.arange(ds.longitude[0], ds.longitude[-1], 0.05)  # 0.4 to 0.05
new_lat = np.arange(ds.latitude[-1], ds.latitude[0], 0.05)
# melakukan re-gridding menggunakan interpolation
ds = ds.interp(latitude=new_lat, longitude=new_lon)

In [13]:
# mengubah data menjadi data tipe dataframa
df = ds.to_dataframe()
# konversi kg/m3 menjadi mikrogram/m3
df ["pm2p5"] = df["pm2p5"] * 1000000000
# menyimpan dataframe menjadi .csv
df.to_csv("pm25_satuan.csv")

In [198]:
import geopandas as gpd
import pandas as pd

In [199]:
# Load shapefile into a GeoDataFrame
gdf_shapefile = gpd.read_file('/Users/roniabuhanif/Downloads/BMKG/data_skripsi/pm25/peta_dki/kota_dkijakarta.shp')

In [200]:
gdf_shapefile.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [201]:
gdf_shapefile.head(5)

,PROVNO,KABKOTNO,PROVINSI,KABKOT,geometry
0,31,01,DKI JAKARTA,KEPULAUAN SERIBU,"MULTIPOLYGON Z (((106.73563 -6.04014 0.00000, ..."
1,31,71,DKI JAKARTA,JAKARTA SELATAN,"POLYGON Z ((106.83399 -6.20492 0.00000, 106.83..."
2,31,72,DKI JAKARTA,JAKARTA TIMUR,"POLYGON Z ((106.94011 -6.15492 0.00000, 106.94..."
3,31,73,DKI JAKARTA,JAKARTA PUSAT,"POLYGON Z ((106.83247 -6.14138 0.00000, 106.83..."
4,31,74,DKI JAKARTA,JAKARTA BARAT,"POLYGON Z ((106.71133 -6.09591 0.00000, 106.71..."


In [202]:
# Load CSV
df_csv = pd.read_csv('pm25_satuan.csv')

In [203]:
df_csv.head(54000)

,time,latitude,longitude,pm2p5
0,2018-01-01,-7.00,106.00,6.207951e-08
1,2018-01-01,-7.00,106.05,6.474401e-08
2,2018-01-01,-7.00,106.10,6.740850e-08
3,2018-01-01,-7.00,106.15,7.007299e-08
4,2018-01-01,-7.00,106.20,7.273749e-08
...,...,...,...,...
53995,2022-12-01,-5.55,107.25,7.825953e-08
53996,2022-12-01,-5.55,107.30,7.837438e-08
53997,2022-12-01,-5.55,107.35,7.848922e-08
53998,2022-12-01,-5.55,107.40,7.860407e-08


sampai sini


In [230]:
# Convert DataFrame to GeoDataFrame
geometry = gpd.points_from_xy(df_csv['longitude'], df_csv['latitude'])
gdf_csv = gpd.GeoDataFrame(df_csv, geometry=geometry)


In [234]:
# spatial join
result = gpd.sjoin(gdf_shapefile, gdf_csv, how='inner', predicate='within')
dissolved_result = result.dissolve(by= 'KABKOT', aggfunc=['min', 'max', 'mean'])
dissolved_result

/var/folders/81/vqbnjx_x20zdr2m7bscdvp5r0000gn/T/ipykernel_11228/2209865297.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  result = gpd.sjoin(gdf_shapefile, gdf_csv, how='inner', predicate='within')


,geometry,"(PROVNO, min)","(PROVNO, max)","(PROVNO, mean)","(KABKOTNO, min)","(KABKOTNO, max)","(KABKOTNO, mean)","(PROVINSI, min)","(PROVINSI, max)","(PROVINSI, mean)",...,"(time, mean)","(latitude, min)","(latitude, max)","(latitude, mean)","(longitude, min)","(longitude, max)","(longitude, mean)","(pm2p5, min)","(pm2p5, max)","(pm2p5, mean)"
KABKOT,,,,,,,,,,,,,,,,,,,,,
